# Russian Billiard
Recreate Method proposed by Grigoriev (2013)

In [4]:
# basic modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import matplotlib.animation as animation
#import plotly.express as px

# spice module
import spiceypy as spice

# astropy and poliastro modules
import astropy.units as u
from astropy import time

import poliastro
from poliastro import iod
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver


gzip was not found on your system! You should solve this issue for astroquery.eso to be at its best!
On POSIX system: make sure gzip is installed and in your path!On Windows: same for 7-zip (http://www.7-zip.org)!


In [5]:
# load spice kernels
spice.furnsh('../spice/naif0012.tls')

# define gravitational parameters
MU = 1.32712440018*10**11   # sun's gravitational parameter [km^3/s^2]
AU = 1.49597870691*10**8    # astronomical unit [AU] to [km]
g0 = 9.80665                # gravitational acceleration [m/s^2]

# define spacecraft parameters
msc_min = 500               # minimum spacecraft mass [kg]
tdep_min = 57023.0          # earliest departure time 00:00:00 January 1, 2015 [MJD]
tdep_max = 61401.0          # latest depature time 24:00:00 December 31, 2025 [MJD]


# conversion of MJD to JD
def MJD2JD(mjd):
    """function converts MJD (Modified Julian Date) to JD (Julian Date)
    Args:
        mjd (float): ephemeris in MJD
    Returns:
        (float): ephemeris equivalent in JD
    """
    jd = mjd + 2400000.5
    return jd


In [8]:
# import ephemerides data from GTOC4 into pandas dataframe
df = pd.read_csv('../gtoc4_problem_data.txt', skiprows=2, delimiter= '\s+',header=None)
df.columns = ['Name','Epoch','a','e','i','LAN','omega','M0']
df.head()

num,tmp = df.shape
print(df.shape)
df.tail(10)

(1436, 8)


,Name,Epoch,a,e,i,LAN,omega,M0
1426,'2008VC',54800.0,1.121759,0.172921,5.724475,218.482074,240.537647,339.437835
1427,'2008VF',54800.0,0.906074,0.325772,26.187102,234.456092,3.245131,204.193374
1428,'2008VG14',54800.0,2.867099,0.564880,10.181283,260.149763,112.941738,12.831283
1429,'2008VJ',54800.0,1.709902,0.467051,25.905756,227.553901,83.294990,51.255520
1430,'2008VU4',54800.0,2.371886,0.770038,11.974151,291.191885,23.168510,23.037338
1431,'2008WB',54800.0,1.371659,0.088384,43.893618,236.939816,163.400090,17.857428
1432,'2008WK',54800.0,1.420894,0.283407,6.366829,61.867245,28.835917,348.715701
1433,'2008WL',54800.0,2.740673,0.644534,6.725985,277.274962,109.877188,7.889516
1434,'2008WM',54800.0,1.073814,0.141095,12.313814,57.730478,84.910270,299.819922
1435,'6344P-L',54800.0,2.804108,0.667083,4.726812,183.611772,234.069655,79.379138
